In [1]:
!nvidia-smi

Thu Nov 26 03:00:53 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!free -m

              total        used        free      shared  buff/cache   available
Mem:          13021         560       10499           0        1960       12207
Swap:             0           0           0


## **What are we going to do:**

- load the dataset from kaggle
- prepare the dataset and build a ``TextDataset``
- load the pre-trained GPT-2 model and tokenizer
- initialize ``Trainer`` with ``TrainingArguments``
- train and save the model
- test the model

In [3]:
!pip install transformers

     |████████████████████████████████| 1.3MB 22.5MB/s 
     |████████████████████████████████| 890kB 48.3MB/s 
     |████████████████████████████████| 1.1MB 52.4MB/s 
     |████████████████████████████████| 2.9MB 49.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=cd784026df86bf294cdac89990274498354e623c53076315b2baff33597667f2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import torch
from transformers import AutoModel, AutoTokenizer

#phobert = AutoModel.from_pretrained("vinai/phobert-base")
custokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
custokenizer.add_tokens('\n')

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


1

In [5]:
len(custokenizer)

64002

In [6]:
train_path = '68cleanregex.txt'
test_path = '68cleanvl_test.txt'

In [12]:
import os
import torch
from torch.utils.data.dataset import Dataset
from transformers.tokenization_utils import PreTrainedTokenizer
from filelock import FileLock
from transformers.utils import logging
from typing import Dict, List, Optional
import pickle
import random
import time

logger = logging.get_logger(__name__)
class CusTextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach
    soon.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        file_path: str,
        block_size: int,
        overwrite_cache=False,
        cache_dir: Optional[str] = None,
    ):
        assert os.path.isfile(file_path), f"Input file path {file_path} not found"

        block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)
        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(
            cache_dir if cache_dir is not None else directory,
            "cached_lm_{}_{}_{}".format(
                tokenizer.__class__.__name__,
                str(block_size),
                filename,
            ),
        )
        lock_path = cached_features_file + ".lock"
        with FileLock(lock_path):

            if os.path.exists(cached_features_file) and not overwrite_cache:
                start = time.time()
                with open(cached_features_file, "rb") as handle:
                    self.examples = pickle.load(handle)
                logger.info(
                    f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
                )

            else:
                logger.info(f"Creating features from dataset file at {directory}")

                self.examples = []
                with open(file_path, encoding="utf-8") as f:
                    text = f.read()

                tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

                # for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                i = 0
                while i < len(tokenized_text) - block_size + 1:
                    inds = tokenized_text[i : i + block_size]
                    # for j in range(0, len(inds)):
                    #     if inds[j]==64001:
                    #         inds = inds[j+1:] #remove the first \n
                    #         break
                    # for j in range(len(inds)-1, 0, -1):
                    #     if inds[j]==64001:
                    #         inds = inds[:j-1] #remove \n
                    #         break
                    i += len(inds)
                    self.examples.append(
                        tokenizer.build_inputs_with_special_tokens(inds)
                    )

                start = time.time()
                with open(cached_features_file, "wb") as handle:
                    pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
                logger.info(
                    "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
                )

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i) -> torch.Tensor:
        return torch.tensor(self.examples[i], dtype=torch.long)

In [9]:
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling, LineByLineWithSOPTextDataset

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = CusTextDataset(
          tokenizer=custokenizer,
          file_path=train_path,
          block_size=256)
     
    test_dataset = CusTextDataset(
          tokenizer=custokenizer,
          file_path=test_path,
          block_size=256)
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=custokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,custokenizer)

In [13]:
len(train_dataset), len(test_dataset)

(1672, 50)

In [14]:
print(custokenizer.decode(train_dataset[0]))

<s> đường thu vàng cả thời gian 
 gió se se lạnh vít làn thu sang 
 đường quê nhạt nắng hanh vàng 
 bao nhiêu kỷ niệm mơ màng trong ta 
 gió thu môi ấm mặn mà 
 ven hồ liễu rủ tóc nhoà điểm sương 
 trời xanh chiều tím quê hương 
 tìm về ký ức sắc hường ngẩn ngơ 
 chông chênh cây trút lá rồi 
 cành ươm mầm nhú khoảng trời nhấp nhô 
 hương quê trên đường ngoại ô 
 thu cùng ta đến bến bờ yêu thương 
 
 trăng là hoa của trời trong 
 sao kia tựa những cánh ong rụng rời 
 gió đàn hưu hắt người ơi 
 mây tơ con nhện rối bời không gian 
 trời cao mây gợn nắng lan 
 gió heo may thổi miên man bên rào 
 vài con bướm trắng lao xao 
 vàng bông hoa cúc cài vào vạt bay 
 thu rồi đồng rải heo may 
 bờ tre ngả úa sương bay là là 
 
 tìm về một thuở hạ xưa 
 bâng khuâng lối nhỏ rào thưa ngày nào 
 tìm vầng trăng rụng cầu ao 
 có con đom đóm đậu vào vai tôi 
 về tìm cái giậu mùng tơi 
 có cô hàng xóm thường chơi bên nhà 
 tìm về một khúc dân ca 
 có đàn cò trắng bay ra cánh đồng 
 tìm về nơi ấy dòng sông 

In [15]:
train_dataset[0].shape

torch.Size([256])

# Initialize `Trainer` with `TrainingArguments` and GPT-2 model

The [Trainer](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer) class provides an API for feature-complete training. It is used in most of the [example scripts](https://huggingface.co/transformers/examples.html) from Huggingface. Before we can instantiate our `Trainer` we need to download our GPT-2 model and create a [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) to access all the points of customization during training. In the `TrainingArguments`, we can define the Hyperparameters we are going to use in the training process like our `learning_rate`, `num_train_epochs`, or  `per_device_train_batch_size`. A complete list can you find [here](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments).

In [16]:
from transformers import Trainer, TrainingArguments, GPT2Config, GPT2LMHeadModel

Define model

In [17]:
pretrainedgpt2 = GPT2LMHeadModel.from_pretrained('gpt2')
# pretrainedgpt2.resize_token_embeddings(len(custokenizer))
pretrainedgpt2

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [18]:
task = {"text-generation": {"do_sample": True, "max_length": 256}} #edit output size

In [19]:
configuration = GPT2Config(vocab_size=64002, n_positions=258, n_ctx=258,
                           task_specific_params=task,
                           eos_token_id=custokenizer.eos_token_id,
                           bos_token_id=custokenizer.bos_token_id, 
                           pad_token_id=custokenizer.pad_token_id,
                           sep_token_id=custokenizer.sep_token_id
                           )
poem = GPT2LMHeadModel(configuration)
poem

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(64002, 768)
    (wpe): Embedding(258, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Layer

In [21]:
poem.config
# poem.resize_token_embeddings(len(custokenizer))
# poem.load_state_dict(pretrainedgpt2.state_dict())

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 258,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 258,
  "pad_token_id": 1,
  "resid_pdrop": 0.1,
  "sep_token_id": 2,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 256
    }
  },
  "vocab_size": 64002
}

In [20]:
from transformers.trainer_callback import TrainerCallback
from transformers import pipeline
class PrinterCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, model=None, **kwargs):
        if int(state.epoch)%10 == 0:
            pipe = pipeline('text-generation', model=model, tokenizer=custokenizer, device=0)
            with open("sample.txt", "a") as f:
                f.write(pipe('<s> tìm về một thuở hạ xưa')[0]['generated_text'])
                f.write("\n===========================================\n")
                f.close()

In [22]:
training_args = TrainingArguments(
    output_dir="gpt2-vnmesepoem", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=200, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    save_steps=10000, # after # steps model is saved 
    save_total_limit = 2, # delete other checkpoints
    warmup_steps=500,    # number of warmup steps for learning rate scheduler
    logging_dir='logs', # directory for storing logs
    logging_steps=500,
    )

In [23]:
trainer = Trainer(
    model=poem,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    prediction_loss_only=True,
    callbacks = [PrinterCallback],
)

/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:281: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


In [24]:
trainer.train()
trainer.save_model()

Step,Training Loss
500,7.771526
1000,6.138330
1500,5.639254
2000,5.267457
2500,4.967254
3000,4.674738
3500,4.403340
4000,4.128566
4500,3.837727
5000,3.558480


KeyboardInterrupt: ignored

In [27]:
from transformers import pipeline

poem = pipeline('text-generation', model="/content/gpt2-vnmesepoem/checkpoint-20000", tokenizer=custokenizer, device=0)

In [28]:
a = poem('<s> mùa xuân')
print(a[0]['generated_text'])

<s> mùa xuân ai cũng ước ao 
 sức khoẻ dồi dào cây cối đơm bông 
 
 mọi người theo dõi chờ trông 
 kết quả trận đấu hợp đồng ra sao 
 dù cho kết quả thế nào 
 niềm tin mãi mãi gửi vào các em 
 những đường truyền bóng đã quen 
 tinh thần quả cảm làm nền vươn xa 
 mong rằng đội bóng nước nhà 
 vượt qua nhật bản kết hoa dâng đời 
 màu cờ sắc áo em ơi 
 lịch sử tổ quốc muôn đời ghi công 
 
 ô kìa xuân chẳng bao xa 
 đào mai hé nụ chan hoà muôn nơi 
 
 hoàng hôn tím cả muộn phiền 
 cánh diều lạc gió rơi miền hoang vu 
 
 người ta ở mãi phương nào 
 đêm nay có thấy thì thào mưa rơi 
 người ta à người ta ơi 
 cho mình được gọi một lời thân thương 
 xa nhau cách mấy chặng đường 
 hình như duyên phận đã vương vào đời 
 mình dù thỉnh thoảng rong chơi 
 nhưng sao quên được sinh nơi quê mùa 
 
 đó là vững chí bền gan 
 khó khăng chẳng ngại gian nan xem thường 
 thông minh sáng tạo kiên cường 
 mốc son lịch sử phi thường việt nam 
 đêm này cả nước hân hoan 
 ta cùng cờ đỏ sao vang tung bay 
 
 đầu 

In [29]:
b = poem('<s> anh')
print(b[0]['generated_text'])

<s> anh hết cả những lời yêu thương 
 khi xưa anh sợ gái mường 
 ngày nay lại thích cơm nương gà đồi 
 nơi này suối khoáng đông vui 
 rượu cần múa xạp núi đồi ngả nghiêng 
 dưới trăng thánh thót cồng chiêng 
 suối trong nước ấm soi miền thiên thai 
 long lanh từng giọt sương cài 
 đậu trên khóe mắt bờ vai trắng ngần 
 ngắm hoa tựa ngắm thiên thần 
 kim bôi nay một ngày xuân giao hòa 
 thương nhau nào quản đường xa 
 vượt trăm cây số để ta thăm nàng 
 gặp đây mường động mường vang 
 hưng yên kết nghĩa họ hàng anh em 
 cùng ai bày tỏ nỗi niềm 
 xin về phố hiến hưng yên một nhà 
 
 chẳng cần má phấn môi son 
 lưng ong áo gụ nước non đong đầy 
 mắt huyền núi ngọc tóc mây 
 trêu lòng quân tử chết ngây nữ kiều 
 
 thanh đoàn tùng đến phạm kha 
 chi nam cò trắng bay qua ngũ hùng 
 tứ cường chi bắc mùa vàng 
 lê hồng đoàn kết hồng quang tân trào 
 lam sơn hương bưởi ngạt ngào 
 vấn vương nhung nhớ sớm nay anh về 
 
 đơn sơ góc bếp mái nhà 
 có em bên cạnh mẹ già cười vui 
 sẻ chia hạnh phúc ng